In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re as re
import contractions
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, precision_score,mean_squared_error
from sklearn.metrics import precision_recall_fscore_support as score

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aakritinag/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aakritinag/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#! pip install bs4 # in case you don't have it installed
# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## Read Data

In [3]:
df = pd.read_table('data.tsv', sep='\t', low_memory=False, on_bad_lines='skip')
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0.0,0.0,N,Y,Five Stars,"Love this, excellent sun block!!",2015-08-31
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0.0,0.0,N,Y,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31
2,US,19242472,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5,0.0,0.0,N,Y,Five Stars,"Great Product, I'm 65 years old and this is al...",2015-08-31
3,US,19551372,R3KSZHPAEVPEAL,B002HWS7RM,255651889,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5,0.0,0.0,N,Y,GOOD DEAL!,I use them as shower caps & conditioning caps....,2015-08-31
4,US,14802407,RAI2OIG50KZ43,B00SM99KWU,116158747,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,This is my go-to daily sunblock. It leaves no ...,2015-08-31


In [4]:
df.reset_index(drop=True, inplace=True)

## Keep Reviews and Ratings

In [5]:
enter = df [['star_rating','review_body']]
enter.head()

,star_rating,review_body
0,5,"Love this, excellent sun block!!"
1,5,The great thing about this cream is that it do...
2,5,"Great Product, I'm 65 years old and this is al..."
3,5,I use them as shower caps & conditioning caps....
4,5,This is my go-to daily sunblock. It leaves no ...


In [6]:
enter['star_rating'].unique()

array(['5', '4', '1', '3', '2', '2015-08-28', '2015-08-16', '2015-08-14',
       '2015-07-27', '2015-07-26', '2015-07-23', '2015-07-22', nan,
       '2015-06-14', '2015-06-02', '2015-04-14', '2015-04-09',
       '2015-04-08', '2015-04-03', '2015-04-02', '2015-04-01',
       '2015-03-31', '2015-03-30', '2015-03-18', '2015-02-28',
       '2015-02-10', '2014-12-30', '2014-12-03', '2014-10-09'],
      dtype=object)

In [7]:
enter['star_rating'] = enter['star_rating'].str.replace(r'\D', '')

/var/folders/m4/yx3n91cs37nf91mrrvjx9s2r0000gn/T/ipykernel_15405/955414285.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  enter['star_rating'] = enter['star_rating'].str.replace(r'\D', '')
/var/folders/m4/yx3n91cs37nf91mrrvjx9s2r0000gn/T/ipykernel_15405/955414285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enter['star_rating'] = enter['star_rating'].str.replace(r'\D', '')


In [8]:
# Drop rows that has NaN values 
enter = enter.dropna()

In [9]:
enter['star_rating'].unique()

array(['5', '4', '1', '3', '2'], dtype=object)

In [10]:
#keep only integers
enter['star_rating'] = enter['star_rating'].astype(int)

/var/folders/m4/yx3n91cs37nf91mrrvjx9s2r0000gn/T/ipykernel_15405/2175660001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enter['star_rating'] = enter['star_rating'].astype(int)


In [11]:
enter.head()

,star_rating,review_body
0,5,"Love this, excellent sun block!!"
1,5,The great thing about this cream is that it do...
2,5,"Great Product, I'm 65 years old and this is al..."
3,5,I use them as shower caps & conditioning caps....
4,5,This is my go-to daily sunblock. It leaves no ...


In [12]:
enter['star_rating'].unique()

array([5, 4, 1, 3, 2])

 ## We form three classes and select 20000 reviews randomly from each class.



In [13]:
def filter(x):
    if (x <= 2):
        return '1'
    if (x == 3):
        return '2'
    if (x > 3) :
        return '3'
enter['category'] = enter['star_rating'].apply(filter)

/var/folders/m4/yx3n91cs37nf91mrrvjx9s2r0000gn/T/ipykernel_15405/2450587933.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enter['category'] = enter['star_rating'].apply(filter)


In [14]:
enter.head()

,star_rating,review_body,category
0,5,"Love this, excellent sun block!!",3
1,5,The great thing about this cream is that it do...,3
2,5,"Great Product, I'm 65 years old and this is al...",3
3,5,I use them as shower caps & conditioning caps....,3
4,5,This is my go-to daily sunblock. It leaves no ...,3


In [15]:
enter = enter.drop('star_rating', axis=1)

In [16]:
enter = enter.groupby('category').apply(lambda s: s.sample(20000))
enter

review_body category
category                                                                    
1        4414357  This pulled out a lot of my hair in very littl...        1
         1066754  Nice design but both light settings are dull. ...        1
         527080         So weird, doesn't even stay out on the nose        1
         2693642  I haven't used it yet, I didn't know it was fr...        1
         3637776  I bought this in July. The light quit working ...        1
...                                                             ...      ...
3        508555                            I love this product !!!!        3
         4038640  I have been using Agadir for a couple of years...        3
         2254554                       Simple, quality, it's great.        3
         3337063  Decent size bottle for the price. This shampoo...        3
         214567   I love these Acrylic Nails! They are perfect f...        3

[60000 rows x 2 columns]

# Data Cleaning



In [17]:
#average length of reviews before cleaning
before_clean = enter['review_body'].str.len().mean()

In [18]:
# convert all reviews into lowercase
enter['review_body'] = enter['review_body'].str.lower()

In [19]:
enter

review_body category
category                                                                    
1        4414357  this pulled out a lot of my hair in very littl...        1
         1066754  nice design but both light settings are dull. ...        1
         527080         so weird, doesn't even stay out on the nose        1
         2693642  i haven't used it yet, i didn't know it was fr...        1
         3637776  i bought this in july. the light quit working ...        1
...                                                             ...      ...
3        508555                            i love this product !!!!        3
         4038640  i have been using agadir for a couple of years...        3
         2254554                       simple, quality, it's great.        3
         3337063  decent size bottle for the price. this shampoo...        3
         214567   i love these acrylic nails! they are perfect f...        3

[60000 rows x 2 columns]

In [20]:
#remove the HTML and URLs from the review
def remove(s):
    s=s.replace(r'<[^<>]*>', '')
    s=s.replace(r'http\S+', '').replace(r'www\S+', '')
    return s
enter['review_body']=enter['review_body'].apply(remove)
enter

review_body category
category                                                                    
1        4414357  this pulled out a lot of my hair in very littl...        1
         1066754  nice design but both light settings are dull. ...        1
         527080         so weird, doesn't even stay out on the nose        1
         2693642  i haven't used it yet, i didn't know it was fr...        1
         3637776  i bought this in july. the light quit working ...        1
...                                                             ...      ...
3        508555                            i love this product !!!!        3
         4038640  i have been using agadir for a couple of years...        3
         2254554                       simple, quality, it's great.        3
         3337063  decent size bottle for the price. this shampoo...        3
         214567   i love these acrylic nails! they are perfect f...        3

[60000 rows x 2 columns]

In [21]:
#Perform contractions on the reviews
enter['review_body'] = enter['review_body'].apply(lambda x: [contractions.fix(word) for word in x.split()])
enter['review_body'] = [' '.join(map(str, l)) for l in enter['review_body']]

In [22]:
#remove non alphabetical characyers and extra spaces
def remove(sentence):
    sentence=re.sub(r'[^a-zA-Z]',' ',sentence)
    return re.sub(' +', ' ',sentence)
enter['review_body'] = enter['review_body'].apply(remove)

In [23]:
enter

review_body category
category                                                                    
1        4414357  this pulled out a lot of my hair in very littl...        1
         1066754  nice design but both light settings are dull r...        1
         527080         so weird does not even stay out on the nose        1
         2693642  i have not used it yet i did not know it was f...        1
         3637776  i bought this in july the light quit working w...        1
...                                                             ...      ...
3        508555                                i love this product         3
         4038640  i have been using agadir for a couple of years...        3
         2254554                        simple quality it is great         3
         3337063  decent size bottle for the price this shampoo ...        3
         214567   i love these acrylic nails they are perfect fo...        3

[60000 rows x 2 columns]

In [24]:
# average length of reviews after cleaning 
after_clean=enter['review_body'].str.len().mean()

# Pre-processing

In [25]:
# average length of reviews before preprocessing 
before_preprocessing = enter['review_body'].str.len().mean()

## remove the stop words 

In [26]:
def remove_non_ascii(text): 
    return ''.join(i for i in text if ord(i)<128) 
 
enter['review_body'] = enter['review_body'].apply(remove_non_ascii) 

In [27]:
# stop = stopwords.words('english')
# enter['review_body'] = enter['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

## perform lemmatization  

In [28]:
lem = WordNetLemmatizer()

enter['review_body']=enter['review_body'].apply(lambda x: " ".join([lem.lemmatize(w) for w in nltk.word_tokenize(x)]))

enter

review_body category
category                                                                    
1        4414357  this pulled out a lot of my hair in very littl...        1
         1066754  nice design but both light setting are dull re...        1
         527080          so weird doe not even stay out on the nose        1
         2693642  i have not used it yet i did not know it wa fr...        1
         3637776  i bought this in july the light quit working w...        1
...                                                             ...      ...
3        508555                                 i love this product        3
         4038640  i have been using agadir for a couple of year ...        3
         2254554                         simple quality it is great        3
         3337063  decent size bottle for the price this shampoo ...        3
         214567   i love these acrylic nail they are perfect for...        3

[60000 rows x 2 columns]

In [29]:
# average length of reviews after preprocessing
after_preprocessing = enter['review_body'].str.len().mean()

In [30]:
print('Before and after preprocessing : '+str(before_preprocessing), ', '+str(after_preprocessing))

Before and after preprocessing : 258.2294 , 253.76626666666667


# TF-IDF Feature Extraction

In [31]:
# from sklearn.feature_extraction.text import TfidfVectorizer
imp = TfidfVectorizer(ngram_range=(1,3))
x = imp.fit_transform(enter['review_body'])

In [32]:
# from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, enter['category'], test_size=0.2, random_state=42)

# Perceptron

In [33]:
from sklearn.linear_model import Perceptron
clf_per = Perceptron(random_state=42, tol=1e-3)
clf_per.fit(X_train,Y_train)

Perceptron(random_state=42)

In [34]:
pred_per= clf_per.predict(X_test)

In [35]:
print(confusion_matrix(Y_test,pred_per))

[[2979  758  226]
 [1126 2288  600]
 [ 218  503 3302]]


In [36]:
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(Y_test, pred_per, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.69      0.75      0.72      3963
     class 2       0.64      0.57      0.61      4014
     class 3       0.80      0.82      0.81      4023

    accuracy                           0.71     12000
   macro avg       0.71      0.71      0.71     12000
weighted avg       0.71      0.71      0.71     12000



In [37]:
precision, recall, fscore, support = score(Y_test, pred_per)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.68910479 0.64468864 0.7999031 ]
recall: [0.75170326 0.57000498 0.82078051]
fscore: [0.71904417 0.60505091 0.81020734]


In [38]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test,pred_per))

Accuracy: 0.7140833333333333


In [39]:
precision, recall, fscore, support = score(Y_test, pred_per)
sum_precision=0
sum_recall=0
sum_f1score=0
for i in range(3):
    print('class '+ str(i+1),'Precision:',precision[i],', Recall:', recall[i], ', F1 score:',fscore[i])
    sum_precision+= precision[i]
    sum_recall+= recall[i]
    sum_f1score+= fscore[i]
print('Averages : ' + str(sum_precision/3)+' , '+str(sum_recall/3)+' , '+str(sum_f1score/3))

class 1 Precision: 0.6891047883414295 , Recall: 0.7517032551097653 , F1 score: 0.719044170890659
class 2 Precision: 0.6446886446886447 , Recall: 0.5700049825610364 , F1 score: 0.6050509057252413
class 3 Precision: 0.7999031007751938 , Recall: 0.8207805120556798 , F1 score: 0.8102073365231259
Averages : 0.7112321779350893 , 0.7141629165754938 , 0.7114341377130087


# SVM

In [40]:
from sklearn.svm import LinearSVC
clf_svm= LinearSVC(random_state=0)
clf_svm.fit(X_train,Y_train)

LinearSVC(random_state=0)

In [41]:
pred_svm=clf_svm.predict(X_test)

In [42]:
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(Y_test, pred_svm, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.73      0.75      0.74      3963
     class 2       0.66      0.63      0.65      4014
     class 3       0.82      0.84      0.83      4023

    accuracy                           0.74     12000
   macro avg       0.74      0.74      0.74     12000
weighted avg       0.74      0.74      0.74     12000



In [43]:
precision, recall, fscore, support = score(Y_test, pred_svm)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.73067454 0.66257188 0.82198444]
recall: [0.74892758 0.63153961 0.84016903]
fscore: [0.73968847 0.64668367 0.83097726]


In [44]:
print("Accuracy:",metrics.accuracy_score(Y_test,pred_svm))

Accuracy: 0.74025


In [45]:
precision, recall, fscore, support = score(Y_test, pred_svm)
sum_precision=0
sum_recall=0
sum_f1score=0
for i in range(3):
    print('class '+ str(i+1),'Precision:',precision[i],', Recall:', recall[i], ', F1 score:',fscore[i])
    sum_precision+= precision[i]
    sum_recall+= recall[i]
    sum_f1score+= fscore[i]
print('Averages : ' + str(sum_precision/3)+' , '+str(sum_recall/3)+' , '+str(sum_f1score/3) )

class 1 Precision: 0.7306745445593303 , Recall: 0.7489275801160736 , F1 score: 0.7396884735202491
class 2 Precision: 0.6625718766335599 , Recall: 0.6315396113602392 , F1 score: 0.6466836734693877
class 3 Precision: 0.8219844357976653 , Recall: 0.8401690280884911 , F1 score: 0.8309772587584511
Averages : 0.7384102856635186 , 0.740212073188268 , 0.739116468582696


# Logistic Regression

In [46]:
from sklearn.linear_model import LogisticRegression

clf_log= LogisticRegression(random_state=42, max_iter=10000)
clf_log.fit(X_train,Y_train)

LogisticRegression(max_iter=10000, random_state=42)

In [47]:
pred_log=clf_log.predict(X_test)

In [48]:
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(Y_test, pred_log, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.73      0.75      0.74      3963
     class 2       0.66      0.65      0.66      4014
     class 3       0.83      0.82      0.83      4023

    accuracy                           0.74     12000
   macro avg       0.74      0.74      0.74     12000
weighted avg       0.74      0.74      0.74     12000



In [49]:
precision, recall, fscore, support = score(Y_test, pred_log)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.72707209 0.66151116 0.83006536]
recall: [0.74817058 0.64997509 0.82078051]
fscore: [0.73747046 0.65569239 0.82539683]


In [50]:
print("Accuracy:",metrics.accuracy_score(Y_test,pred_log))

Accuracy: 0.7396666666666667


In [51]:
precision, recall, fscore, support = score(Y_test, pred_log)
sum_precision=0
sum_recall=0
sum_f1score=0
for i in range(3):
    print('class '+ str(i+1),'Precision:',precision[i],', Recall:', recall[i], ', F1 score:',fscore[i])
    sum_precision+= precision[i]
    sum_recall+= recall[i]
    sum_f1score+= fscore[i]
print('Averages : ' + str(sum_precision/3)+' , '+str(sum_recall/3)+' , '+str(sum_f1score/3) )

class 1 Precision: 0.7270720941638058 , Recall: 0.7481705778450669 , F1 score: 0.7374704638726527
class 2 Precision: 0.6615111561866126 , Recall: 0.6499750871948181 , F1 score: 0.6556923850213622
class 3 Precision: 0.8300653594771242 , Recall: 0.8207805120556798 , F1 score: 0.8253968253968255
Averages : 0.7395495366091809 , 0.7396420590318549 , 0.7395198914302802


# Naive Bayes

In [52]:
from sklearn.naive_bayes import MultinomialNB

clf_naive = MultinomialNB()
clf_naive.fit(X_train, Y_train)

MultinomialNB()

In [53]:
pred_naive = clf_naive.predict(X_test)

In [54]:
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(Y_test, pred_naive, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.73      0.74      0.73      3963
     class 2       0.61      0.71      0.66      4014
     class 3       0.89      0.74      0.81      4023

    accuracy                           0.73     12000
   macro avg       0.74      0.73      0.73     12000
weighted avg       0.74      0.73      0.73     12000



In [55]:
print("Accuracy:",metrics.accuracy_score(Y_test,pred_naive))

Accuracy: 0.72825


In [56]:
precision, recall, fscore, support = score(Y_test, pred_naive)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.72704715 0.61267454 0.89200603]
recall: [0.73933888 0.7105132  0.73502361]
fscore: [0.7331415  0.6579767  0.80594167]


In [57]:
precision, recall, fscore, support = score(Y_test, pred_naive)
sum_precision=0
sum_recall=0
sum_f1score=0
for i in range(3):
    print('class '+ str(i+1),'Precision:',precision[i],', Recall:', recall[i], ', F1 score:',fscore[i])
    sum_precision+= precision[i]
    sum_recall+= recall[i]
    sum_f1score+= fscore[i]
print('Averages : ' + str(sum_precision/3)+' , '+str(sum_recall/3)+' , '+str(sum_f1score/3))

class 1 Precision: 0.7270471464019851 , Recall: 0.7393388846833208 , F1 score: 0.7331414988114601
class 2 Precision: 0.6126745435016112 , Recall: 0.7105132037867464 , F1 score: 0.6579766985811513
class 3 Precision: 0.8920060331825038 , Recall: 0.7350236142182451 , F1 score: 0.8059416734805125
Averages : 0.7439092410287 , 0.7282919008961041 , 0.7323532902910412
